<a href="https://colab.research.google.com/github/fatfrogfrog/6502cpu/blob/main/cpu6502.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:

code = '''
class CPU6502:
    def __init__(self):
        self.A = 0x00
        self.X = 0x00
        self.Y = 0x00
        self.PC = 0x0000
        self.SP = 0xFD
        self.P = 0x24 #NV-BDIZC
        self.memory = [0x00] * 0x10000

        # Initialize opcode map
        self.opcode_map = {
            0xA9: self.LDA_imm,
            0xAA: self.TAX,       # TAX: Transfer Accumulator to X Register
            0xE8: self.INX,       # 加入 INX 指令支援
            0x8D: self.STA_absolute,   # Store value to any memory address
            0x4C: self.JMP_absolute,   # Jump to any address
            0xC9: self.CMP_imm,     # Compare with A, show result in P
            0xF0: self.BEQ_relative,   # Using zero flag to decide branch
            0xD0: self.BNE_relative,    # Branch not equal
            0xA2: self.LDX_imm,      # Load X register with immediate value
            0xA0: self.LDY_imm,      # Load Y register with immediate value
            0xC8: self.INY,       # Y register += 1
            0x30: self.BMI_relative,   # Negative flag  = 1, jump
            0x29: self.AND_imm      # A = A & value, bits mask

        }

    def reset(self):
        lo = self.memory[0xFFFC]
        hi = self.memory[0xFFFD]
        self.PC = (hi << 8) | lo # left shift 8 bits combine to 16 bit address

    def read(self, addr):
        return self.memory[addr]

    def write(self, addr, value):
        self.memory[addr] = value & 0xFF

    def set_zero_and_negative_flags(self, value):
        if value == 0:
            self.P |= 0x02  # Set Zero flag
        else:
            self.P &= ~0x02  # Clear Zero flag

        if value & 0x80:
            self.P |= 0x80  # Set Negative flag
        else:
            self.P &= ~0x80  # Clear Negative flag

    def step(self):
        opcode = self.read(self.PC)
        self.PC += 1
        if opcode in self.opcode_map:
            self.opcode_map[opcode]()
        else:
            raise NotImplementedError(f"Opcode {hex(opcode)} not implemented.")

    def LDA_imm(self): #immediate put value into register
        value = self.read(self.PC)
        self.PC += 1
        self.A = value
        self.set_zero_and_negative_flags(value)

    def TAX(self):
        self.X = self.A
        self.set_zero_and_negative_flags(self.X)

    def INX(self):
      if self.X != 0xFF:
        self.X += 1
      else:
        self.X = 0x00
      self.set_zero_and_negative_flags(self.X)

    def STA_absolute(self):
      lo = self.read(self.PC)
      self.PC += 1
      hi = self.read(self.PC)
      self.PC += 1
      addr = (hi << 8) | lo
      self.write(addr, self.A)

    def JMP_absolute(self):
      lo = self.read(self.PC)
      self.PC += 1
      hi = self.read(self.PC)
      self.PC += 1
      addr = (hi << 8) | lo
      self.PC = addr

    def CMP_imm(self):
      value = self.read(self.PC)
      self.PC += 1
      result = self.A - value

      # Set Carry if A >= value
      if self.A >= value:
          self.P |= 0x01  # Carry
      else:
          self.P &= ~0x01

      # Set Zero if A == value
      if self.A == value:
          self.P |= 0x02
      else:
          self.P &= ~0x02

      # Set Negative based on result
      if result & 0x80:
          self.P |= 0x80
      else:
          self.P &= ~0x80

    def BEQ_relative(self):
      offset = self.read(self.PC) # 設定偏移量
      self.PC += 1
      if self.P & 0x02:  # Zero flag is set
          if offset < 0x80:
              self.PC += offset
          else:
              self.PC -= (0x100 - offset)  # overflow 二補數轉換

    def BNE_relative(self):
      offset = self.read(self.PC)
      self.PC += 1
      if not (self.P & 0x02):  # Zero flag 為 0
          if offset < 0x80:
              self.PC += offset
          else:
              self.PC -= (0x100 - offset)

    def LDX_imm(self):
      value = self.read(self.PC)
      self.PC += 1
      self.X = value
      self.set_zero_and_negative_flags(self.X)

    def LDY_imm(self):
      value = self.read(self.PC)
      self.PC += 1
      self.Y = value
      self.set_zero_and_negative_flags(self.Y)

    def INY(self):
      self.Y = (self.Y + 1) & 0xFF
      self.set_zero_and_negative_flags(self.Y)

    def BMI_relative(self):
      offset = self.read(self.PC)
      self.PC += 1
      if self.P & 0x80:  # N flag set
          if offset < 0x80:
              self.PC += offset
          else:
              self.PC -= (0x100 - offset)

    def AND_imm(self):
      value = self.read(self.PC)
      self.PC += 1
      self.A = self.A & value
      self.set_zero_and_negative_flags(self.A)





'''
with open("cpu6502.py", "w") as f:
    f.write(code)


In [10]:
# 強制重新載入 cpu6502.py
import importlib
import cpu6502 #import cpu6502.py
importlib.reload(cpu6502) #reload cpu6502.py
from cpu6502 import CPU6502 #import class CPU6502, kind of oop in C++

#======================== Pattern setting ================================#
def run_test(memory_setup, expected_state, debug=False):
    cpu = CPU6502()
    cpu.memory[0xFFFC] = 0x00
    cpu.memory[0xFFFD] = 0x80
    cpu.reset()

    # 載入測試程式碼到記憶體
    for addr, value in memory_setup.items():
        cpu.memory[addr] = value
        #print(hex(cpu.memory[addr]))

    # 預設跑最多20步防止無限 loop
    for _ in range(6):
        if debug:
            print(f"PC={hex(cpu.PC)}, A={hex(cpu.A)}, X={hex(cpu.X)}, Y={hex(cpu.Y)}, P={bin(cpu.P)}")
        cpu.step()

    # 驗證結果
    for reg, val in expected_state.items():
        actual = getattr(cpu, reg)
        assert actual == val, f"❌ {reg} expected {hex(val)}, got {hex(actual)}"
    print("✅ 測試通過")

#======================== Pattern setting ================================#
#=============================== Memory ===================================#
memory = {
    0x8000: 0xA2,  # LDX #$0F
    0x8001: 0x0F,
    0x8002: 0xA0,  # LDY #$FE
    0x8003: 0xFE,
    0x8004: 0xC8,  # INY       (Y=FF → N flag = 1)
    0x8005: 0x30,  # BMI +2
    0x8006: 0x02,
    0x8007: 0xA9,  # LDA #$00  <- 會被跳過
    0x8008: 0xFF,
    0x8009: 0xA9,  # LDA #$AA
    0x800A: 0xAA,
    0x800B: 0x29,  # AND #$0F
    0x800C: 0x0F
}
# golden
expected = {
    'X': 0x0F,     # LDX 成功
    'Y': 0xFF,     # INY 成功
    'A': 0x0A,     # 0xAA AND 0x0F = 0x0A
}

run_test(memory, expected, debug=True)

#=============================== Memory ===================================#

PC=0x8000, A=0x0, X=0x0, Y=0x0, P=0b100100
PC=0x8002, A=0x0, X=0xf, Y=0x0, P=0b100100
PC=0x8004, A=0x0, X=0xf, Y=0xfe, P=0b10100100
PC=0x8005, A=0x0, X=0xf, Y=0xff, P=0b10100100
PC=0x8009, A=0x0, X=0xf, Y=0xff, P=0b10100100
PC=0x800b, A=0xaa, X=0xf, Y=0xff, P=0b10100100
✅ 測試通過
